## Metric-Visualizer Cola-lab Tutorial

In [5]:
!pip install -e ..
# !pip install metric_visualizer

### Metrics Logging
This example shows how to log metrics into metric-visualizer

In [1]:
import os
import random

import findfile

from metric_visualizer import MetricVisualizer
import numpy as np

MV = MetricVisualizer(name="example", trial_tag="Model")

# 随机指标模拟
repeat = 100  # number of repeats of each trial i.e., 每个实验模型/算法/目标/etc的重复次数
metric_num = 2  # number of metrics in each trial  i.e., 每个模型/算法/目标/etc的指标总数

trial_names = ["NSGA-II", "NSGA-III", "MOEA/D"]  # fake trial names

for r in range(repeat):  # repeat the experiments to plot violin or box figure
    metrics = [
        (np.random.random() + n + (1 if random.random() > 0.5 else -1))
        for n in range(metric_num)
    ]  # n is metric scale factor
    for n_trial in range(len(trial_names)):
        for i, m in enumerate(metrics):
            MV.log_metric(
                trial_name="Trial-{}".format(n_trial),
                metric_name="metric{}".format(i + 1),
                value=m * random.random(),
            )  # add metric by custom name and value

### Metrics Summary
This example shows how to view simple metrics statistics

In [3]:
# MV.remove_outliers()  # remove outliers for each metric, do not use this unless you have to

MV.summary(save_path="./example.summary.txt", no_print=False)

MV.to_execl(save_path="example.xlsx")  # save summary to excel
# other usages
# MV.to_csv()
# MV.to_txt()
# MV.to_json()
# MV.to_latex()

### Metrics Visualization
This example shows how to visualize and compare the performance between different trials

#### Box Plot by Tikz

In [3]:
fig_path = MV.box_plot(
    engine="tikz",
    save_path="box_plot_by_trial.tex",
)

#### Important: Reformat to Cola-lab Template
**Tip: Use default matplotlib setting and fine-tune the tikz styles will be better**
Step1: Find a tikz plot template
Step2: Get a tex plot file to reformat
Step3: Fine-tune the plot tikz style settings
Step4: Reformat
Here is the full example:

In [2]:
from metric_visualizer import reformat_tikz_format_for_colalab

# Step1: Let us use the template ./delta_hv.tex (Please find a suitable template based on your data)
tex_template_path = "./delta_hv.tex"

# Step2: Get a tex plot file
fig_path = MV.box_plot(
    engine="tikz",
    save_path="box_plot_by_trial.tex",
    no_overlap=True,  # this will separate different metrics
)

# Step3: Fine-tune the plot tikz style settings, these style settings will be written into tikz source code. So only tikz grammar is allowed
# e.g.,
style_settings = {
    "xtick": {0, 1, 2},  # to be written as xtick = {0,1,2} in .tex file
    "font": r"\LARGE",  # to be written as font=\LARGE in .tex file
    "legend pos": "south east",  # to be written as legend pos=south east in .tex file
    "line width": "2pt",
    "xlabel": "Algorithm",
    "ylabel": "HV",
}

# Step4: Reformat
pdf_path = reformat_tikz_format_for_colalab(
    template=tex_template_path,  # can be a file path or text source texts
    tex_src_to_format=fig_path,  # can be a file path or text source texts
    output_path=fig_path,  # can be a file path
    style_settings=style_settings,
)
os.system('xdg-open "{}"'.format(pdf_path))  # for linux
os.system(r'explorer "file:{}"'.format(pdf_path))  # for windows

#### Box Plot by Matplotlib
You need to install latex and compile the tex source file to get the tikz plots

In [6]:
tikz_file_path = MV.box_plot(
    show=True,
    save_path="box_plot_by_trial.png",
)
MV.compile_tikz(crop=True, clean=True)

#### You can also plot by metrics
All features are available for metric-based visualizations

In [7]:
fig_path = MV.box_plot(
    by="metric",
    show=True,
    save_path="box_plot_by_trial.png",
)

#### Other Available Plots

In [8]:
MV.box_plot(by="trial", show=True)
MV.violin_plot(by="trial", show=True)
MV.bar_plot(by="trial", show=True)
MV.bar_plot(by="trial", show=True)
MV.trajectory_plot(by="trial", show=True)
MV.sk_rank_plot(plot_type="box", show=True)
MV.sk_rank_plot(plot_type="violin", show=True)
MV.a12_bar_plot(show=True)


MV.summary(dump_path=os.getcwd())
MV.box_plot(
    by="trial",
    engine="tikz",
    show=True,
    save_path="box_plot_by_trial.tex",
)
MV.violin_plot(
    by="trial",
    engine="tikz",
    show=True,
    save_path="violin_plot_by_trial.tex",
)
MV.bar_plot(
    by="trial",
    engine="tikz",
    show=True,
    save_path="bar_plot_by_trial.tex",
)
MV.bar_plot(
    by="trial",
    engine="tikz",
    show=True,
    save_path="bar_plot_by_trial.tex",
)
MV.trajectory_plot(
    by="trial",
    engine="tikz",
    show=True,
    save_path="trajectory_plot_by_trial.tex",
)
MV.sk_rank_plot(
    plot_type="box",
    engine="tikz",
    show=True,
    save_path="sk_rank_plot_box.tex",
)
MV.sk_rank_plot(
    plot_type="violin",
    engine="tikz",
    show=True,
    save_path="sk_rank_plot_violin.tex",
)
MV.a12_bar_plot(
    plot_type="violin",
    engine="tikz",
    show=True,
    save_path="sk_rank_plot_violin.tex",
)

### Wilcoxon Rank Test (Mann Whitney U test)
This example shows how to get the rank test results

#### Rank by Trial

In [10]:
rank_test_results = MV.rank_test_by_trail(
    trial="Trial-1", rank_type="two-sided"
)  # rank_type = {'two-sided', 'less', 'greater'}, optional. i.e., the alternative param in scipy.stats.ranksum
print(rank_test_results)

In [9]:
rank_test_results = MV._rank_test_by_trial(
    rank_type="two-sided"
)  # his will give you all rank test results
print(rank_test_results)

#### Rank by Metric

In [9]:
rank_test_results = MV.rank_test_by_metric(
    rank_type="greater"
)  # rank_type = {'two-sided', 'less', 'greater'}, optional. i.e., the alternative param in scipy.stats.ranksum
print(rank_test_results)
# rank_test_results = MV._rank_test_by_metric(rank_type="greater")

### Load Metric-Visualizer File Anywhere
You can load a ".mv" file or a list of ".mv" files to merge example results, this is useful when you have to run experiments iteratively.

In [11]:
mvs = findfile.find_cwd_files(
    [".mv"], exclude_key=[".pdf", "png", ".aux", ".log", ".txt"]
)
loaded_mv = MetricVisualizer.load(mvs)

loaded_mv.summary()

### Load Metric-Visualizer File from Console or Bash

```bash
mvis *.mv
```